<a href="https://colab.research.google.com/github/DallasAutumn/datawhale_salons/blob/master/PyTorch_s8_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision import datasets, transforms


import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

# Hyper params
EPOCHS = 1
BATCH_SIZE = 64
TIME_STEP = 28 # 28步中每步读取一行
INPUT_SIZE = 28 # 每行读取28个像素点
TEST_SIZE = 2000
LR = 0.01

In [0]:
train_data = datasets.MNIST(root='./mnist', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_data = datasets.MNIST(root='./mnist', train=False, transform=transforms.ToTensor())
with torch.no_grad():
    test_x = Variable(test_data.data).type(torch.FloatTensor)[:TEST_SIZE] / 255. # normalization
    test_y = test_data.targets.numpy().squeeze()[:TEST_SIZE]

In [21]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.rnn = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=64,
            num_layers=2,
            batch_first=True,
        )

        self.out = nn.Linear(64, 10)
    
    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :]) # batch_size, time_step, input_size
        return out

rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, num_layers=2, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [23]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    print(f"# epoch {epoch} ...")
    for step, (batch_x, batch_y) in enumerate(train_loader):
        b_x = Variable(batch_x.view(-1, 28, 28)) # Tensor(64, 28, 28)
        b_y = Variable(batch_y) # Tensor(64)
        output = rnn(b_x)
        #print(b_x.size(), b_y.size())
        loss = loss_func(output, b_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_output = rnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
            accuracy = sum(pred_y == test_y) / test_y.size
            print("train loss: %.4f" % loss.item(), "test accuracy: %.2f" % accuracy)

test_output = rnn(test_x[:10].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print('prediction', pred_y, 'real number', test_y[:10])

# epoch 0 ...
train loss: 0.3076 test accuracy: 0.86
train loss: 0.1935 test accuracy: 0.90
train loss: 0.4376 test accuracy: 0.88
train loss: 0.2455 test accuracy: 0.91
train loss: 0.1154 test accuracy: 0.92
train loss: 0.1830 test accuracy: 0.94
train loss: 0.1466 test accuracy: 0.94
train loss: 0.3069 test accuracy: 0.95
train loss: 0.0890 test accuracy: 0.95
train loss: 0.0550 test accuracy: 0.96
train loss: 0.1256 test accuracy: 0.96
train loss: 0.4467 test accuracy: 0.93
train loss: 0.3135 test accuracy: 0.95
train loss: 0.1338 test accuracy: 0.97
train loss: 0.1523 test accuracy: 0.95
train loss: 0.3754 test accuracy: 0.95
train loss: 0.1431 test accuracy: 0.96
train loss: 0.1362 test accuracy: 0.95
train loss: 0.0649 test accuracy: 0.96
prediction [7 2 1 0 4 1 4 9 5 9] real number [7 2 1 0 4 1 4 9 5 9]
